In [4]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer, DPRQuestionEncoder, DPRQuestionEncoderTokenizer
import numpy as np

/home/ubuntu/miniconda3/envs/qatask/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
passage_encoder = "voidful/dpr-ctx_encoder-bert-base-multilingual"
query_encoder = "voidful/dpr-question_encoder-bert-base-multilingual"
context_encoder = DPRContextEncoder.from_pretrained(passage_encoder).cuda()
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained(passage_encoder)
question_encoder = DPRQuestionEncoder.from_pretrained(query_encoder).cuda()
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(query_encoder)

Some weights of the model checkpoint at voidful/dpr-ctx_encoder-bert-base-multilingual were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.weight', 'ctx_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoder

In [87]:
from transformers import pipeline as _pipeline
from collections import Counter
from torch.utils.data import DataLoader, Dataset
pipeline = _pipeline('question-answering', 
        model="nguyenvulebinh/vi-mrc-large", 
        tokenizer="nguyenvulebinh/vi-mrc-large", 
        device="cuda:0", 
        batch_size=50)
# pipeline = _pipeline(
#     "question-answering",
#     model="mrm8488/bert-multi-cased-finetuned-xquadv1",
#     tokenizer="mrm8488/bert-multi-cased-finetuned-xquadv1",
#     device="cuda:0",
#     batch_size=50
# )
class ListDataset(Dataset):
    def __init__(self, original_list):
        self.original_list = original_list

    def __len__(self):
        return len(self.original_list)

    def __getitem__(self, i):
        return self.original_list[i]

In [39]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer_trans = AutoTokenizer.from_pretrained("VietAI/envit5-translation")
model_trans = AutoModelForSeq2SeqLM.from_pretrained("VietAI/envit5-translation").cuda()
def translate(text):
    outputs = model_trans.generate(tokenizer_trans(text, return_tensors="pt", padding=True).input_ids.cuda(), max_length=512)
    en_text = tokenizer_trans.batch_decode(outputs, skip_special_tokens=True)
    return en_text[0][4:]

In [91]:
question = "Hang động nào ở Việt Nam là hang động tự nhiên lớn nhất thế giới?"
contexts =  [
                "Hang Hươu, Hang Hươu Hang Hươu () là một hang ở Borneo, Malaysia, trong Vườn quốc gia Gunung Mulu, một di sản thế giới được UNESCO công nhận. Hang Hươu có chiều cao 196.64 m, rộng 90 m, dài 2 km, từng được xem là hang lớn nhất thế giới cho đến thời điểm năm 2009 khi Hiệp hội Hang động Hoàng gia Anh công bố hang Sơn Đoòng trong Vườn quốc gia Phong Nha-Kẻ Bàng ở Quảng Bình, Việt Nam, là hang có chiều rộng 200 mét, cao hơn 150 mét, dài ít nhất gần 9 km là hang động tự nhiên lớn nhất thế giới. ",
                "Hang Sơn Đoòng, Hang Sơn Đoòng Hang Sơn Đoòng là một hang động tự nhiên tại xã Tân Trạch, huyện Bố Trạch, tỉnh Quảng Bình, Việt Nam. Hang Sơn Đoòng được coi là hang động tự nhiên lớn nhất thế giới đã biết.",
                "Hiệp hội Hang động Hoàng gia Anh, Ngày 22/4/2009, Hiệp hội hang động Hoàng gia Anh công bố trước ủy ban nhân dân tỉnh Quảng Bình phát hiện hang động lớn nhất thế giới tại Việt Nam. Hang được đặt tên Sơn Đoòng, nằm cách đường Hồ Chí Minh tuyến phía Tây 9,5 km, thuộc di sản thiên nhiên thế giới Phong Nha - Kẻ Bàng. Đoàn thám hiểm mới khám phá được 6,5 km do chưa có điều kiện kỹ thuật tốt nên chưa kết luận khoa học về độ dài của hang động mới này.",
                "Quảng Bình, Hang Sơn Đoòng Là hang động tự nhiên lớn nhất thế giới,là kỳ quan thiên nhiên và địa chất của nhân loại,thuộc quần thể hang động Phong Nha - Kẻ Bàng, tọa lạc tại xã Sơn Trạch, huyện Bố Trạch, tỉnh Quảng Bình nước ta. Cho tới nay, đây luôn được coi là hang động tự nhiên lớn nhất thế giới từng được phát hiện.Kích thước của hang Sơn Đoòng rất lớn với chiều dài ít nhất là 5 km, tương đương sức chứa khoảng 68 chiếc máy bay Boeing 777. Thậm chí, con số này có thể lớn hơn bởi theo các nhà khoa học, những phương tiện hiện đại nhất ngày nay cũng chưa khám phá được hết chiều dài thực sự của hang động này.Trong hang Sơn Đoòng, các chuyên gia phát hiện những vị trí có kích thước rất lớn.",
                "Hang, 4/2009 nhóm thám hiểm thuộc Hiệp hội Hang động Hoàng gia Anh công bố phát hiện mới nhất về hang động lớn nhất thế giới Sơn Đoòng, ở di sản thế giới Vườn quốc gia Phong Nha-Kẻ Bàng, tỉnh Quảng Bình, Việt Nam. (hiện đang chờ cập nhật những thông số đo đạc cụ thể). Trước đó giữ kỉ lục này là hang Deer ở Vườn quốc gia Gunung Mulu (Sarawak, Borneo, Malaysia), bề ngang của hang xấp xỉ 600 m và chiều cao từ sàn lên trần hang là 400 m.",
                "Cửu Hương, Các hang đá, thạch nhũ trong hang được đặt tên, khiến cho các khối đá trở nên có hồn và sinh động. Tổng cộng có trên 100 hang động lớn nhỏ và là quần thể hang động lớn nhất Trung Quốc có diện tích 170 km2 nằm ở độ cao từ 1.750m – 1.900 m so với mặt nước biển, nhiệt độ bình quân năm là 14,6oC. Theo kết luận của các nhà hang động học trên Thế giới, quần thể hang động Củu Hương được hình thành từ 600 triệu năm về trước, là một hang động điển hình của châu Âu và trên thế giới chưa nơi nào có. ",
                "Hang động Việt Nam, Hệ thống hang động ở Việt Nam thường là các hang động nằm trong các vùng núi đá vôi có kiểu địa hình karst rất phát triển. Ba di sản thiên nhiên thế giới của Việt Nam là vịnh Hạ Long, Vườn quốc gia Phong Nha - Kẻ Bàng và quần thể danh thắng Tràng An đều là những danh thắng có những hang động karst nổi tiếng.  Tổng quan.",
                "Hang động Việt Nam, Hang động Việt Nam Hang động Việt Nam bao gồm hệ thống các hang và các động trên địa bàn Việt Nam, chủ yếu nằm ở nửa phía bắc của đất nước này do tập trung nhiều dãy núi đá vôi. Hang thường được hiểu là khoảng trống sâu tự nhiên hay được đào vào trong đất, trong đá còn động là hang rộng ăn sâu vào trong núi.",
                "Vườn quốc gia Phong Nha – Kẻ Bàng, Vườn quốc gia này được thiết lập để bảo vệ một trong hai vùng các-xtơ lớn nhất thế giới với khoảng 300 hang động và bảo tồn hệ sinh thái bắc Trường Sơn ở khu vực Bắc Trung Bộ Việt Nam. Đặc trưng của vườn quốc gia này là các kiến tạo đá vôi, 300 hang động, các sông ngầm và hệ động thực vật quý hiếm nằm trong Sách đỏ Việt Nam và Sách đỏ thế giới. Các hang động ở đây có tổng chiều dài khoảng hơn 80 km nhưng các nhà thám hiểm hang động Anh và Việt Nam mới chỉ thám hiểm 20 km, trong đó 17 km ở khu vực Phong Nha và 3 km ở khu vực Kẻ Bàng.",
                "Du lịch Việt Nam, Hang động Việt Nam chủ yếu nằm ở nửa phía bắc của đất nước này do tập trung nhiều dãy núi đá vôi. Hệ thống hang động ở Việt Nam thường là các hang động nằm trong các vùng núi đá vôi có kiểu địa hình karst rất phát triển. Ba di sản thiên nhiên thế giới của Việt Nam là vịnh Hạ Long, Vườn quốc gia Phong Nha - Kẻ Bàng và quần thể danh thắng Tràng An đều là những danh thắng có những hang động nổi tiếng."
            ]

In [101]:
top_k = 5
question_ids = question_tokenizer.encode(translate(question), max_length=256, return_tensors='pt', truncation=True).cuda()
question_embeddings = question_encoder(question_ids).pooler_output.detach().cpu().numpy()
contexts_ids = context_tokenizer([translate(context) for context in contexts], return_tensors='pt', padding="longest", truncation=True, add_special_tokens=True, max_length=512)
contexts_embeddings = context_encoder(contexts_ids["input_ids"].cuda()).pooler_output.detach().cpu().numpy()
scores = np.matmul(question_embeddings, contexts_embeddings.T)
print(scores)
candidate_ids = np.argsort(scores)[0][-top_k:][::-1]
candidate_passages = [contexts[i] for i in candidate_ids]
print(candidate_passages)

[[16.255587 16.507725 16.536337 15.316751 16.203024 15.630155 18.62155
  17.613338 16.906584 19.690176]]
['Du lịch Việt Nam, Hang động Việt Nam chủ yếu nằm ở nửa phía bắc của đất nước này do tập trung nhiều dãy núi đá vôi. Hệ thống hang động ở Việt Nam thường là các hang động nằm trong các vùng núi đá vôi có kiểu địa hình karst rất phát triển. Ba di sản thiên nhiên thế giới của Việt Nam là vịnh Hạ Long, Vườn quốc gia Phong Nha - Kẻ Bàng và quần thể danh thắng Tràng An đều là những danh thắng có những hang động nổi tiếng.', 'Hang động Việt Nam, Hệ thống hang động ở Việt Nam thường là các hang động nằm trong các vùng núi đá vôi có kiểu địa hình karst rất phát triển. Ba di sản thiên nhiên thế giới của Việt Nam là vịnh Hạ Long, Vườn quốc gia Phong Nha - Kẻ Bàng và quần thể danh thắng Tràng An đều là những danh thắng có những hang động karst nổi tiếng.  Tổng quan.', 'Hang động Việt Nam, Hang động Việt Nam Hang động Việt Nam bao gồm hệ thống các hang và các động trên địa bàn Việt Nam, chủ yế

In [102]:
from underthesea import word_tokenize, text_normalize, sent_tokenize
from rank_bm25 import BM25Okapi
def sieve(passages, question):
    question = word_tokenize(text_normalize(question))
    tokenized_passages = []
    for passage in passages:
        sentences = sent_tokenize(passage)
        cleaned_sentences = [text_normalize(sent) for sent in sentences]
        cleaned_sentences = [sent for sent in sentences]
        tokenized_passage = []
        for sent in cleaned_sentences:
            tokenized_passage.extend(word_tokenize(sent))
        tokenized_passages.append(tokenized_passage)
    bm25 = BM25Okapi(tokenized_passages)
    topk_passages = bm25.get_top_n(question, passages, n=3)
    scores = bm25.get_scores(question)
    return topk_passages, scores
topk_bm25, scores_bm25 = sieve(contexts, question)

In [103]:
topk_bm25, scores_bm25

(['Cửu Hương, Các hang đá, thạch nhũ trong hang được đặt tên, khiến cho các khối đá trở nên có hồn và sinh động. Tổng cộng có trên 100 hang động lớn nhỏ và là quần thể hang động lớn nhất Trung Quốc có diện tích 170 km2 nằm ở độ cao từ 1.750m – 1.900 m so với mặt nước biển, nhiệt độ bình quân năm là 14,6oC. Theo kết luận của các nhà hang động học trên Thế giới, quần thể hang động Củu Hương được hình thành từ 600 triệu năm về trước, là một hang động điển hình của châu Âu và trên thế giới chưa nơi nào có. ',
  'Hang Hươu, Hang Hươu Hang Hươu () là một hang ở Borneo, Malaysia, trong Vườn quốc gia Gunung Mulu, một di sản thế giới được UNESCO công nhận. Hang Hươu có chiều cao 196.64 m, rộng 90 m, dài 2 km, từng được xem là hang lớn nhất thế giới cho đến thời điểm năm 2009 khi Hiệp hội Hang động Hoàng gia Anh công bố hang Sơn Đoòng trong Vườn quốc gia Phong Nha-Kẻ Bàng ở Quảng Bình, Việt Nam, là hang có chiều rộng 200 mét, cao hơn 150 mét, dài ít nhất gần 9 km là hang động tự nhiên lớn nhất t

In [104]:
candidate_passages = candidate_passages+topk_bm25


In [105]:
prepared = [{"question": question, "context": passage} for passage in candidate_passages]
prepared_dataset = ListDataset(prepared)
predicted = []
print(prepared)
for batch in DataLoader(prepared_dataset, batch_size=30):
    predicted_batch = pipeline(batch)
    predicted.extend(predicted_batch)
print("="*200)
print(predicted)
passage_scores = [scores[0][i] for i in candidate_ids] + [scores_bm25[j] for j in range(len(topk_bm25))]
total_scores = np.array(passage_scores) + np.array([p["score"]*100 for p in predicted])
print("combined scores", total_scores)
best_id = np.argmax(total_scores)
# best answer
print(predicted[best_id]["answer"])


[{'question': 'Hang động nào ở Việt Nam là hang động tự nhiên lớn nhất thế giới?', 'context': 'Du lịch Việt Nam, Hang động Việt Nam chủ yếu nằm ở nửa phía bắc của đất nước này do tập trung nhiều dãy núi đá vôi. Hệ thống hang động ở Việt Nam thường là các hang động nằm trong các vùng núi đá vôi có kiểu địa hình karst rất phát triển. Ba di sản thiên nhiên thế giới của Việt Nam là vịnh Hạ Long, Vườn quốc gia Phong Nha - Kẻ Bàng và quần thể danh thắng Tràng An đều là những danh thắng có những hang động nổi tiếng.'}, {'question': 'Hang động nào ở Việt Nam là hang động tự nhiên lớn nhất thế giới?', 'context': 'Hang động Việt Nam, Hệ thống hang động ở Việt Nam thường là các hang động nằm trong các vùng núi đá vôi có kiểu địa hình karst rất phát triển. Ba di sản thiên nhiên thế giới của Việt Nam là vịnh Hạ Long, Vườn quốc gia Phong Nha - Kẻ Bàng và quần thể danh thắng Tràng An đều là những danh thắng có những hang động karst nổi tiếng.  Tổng quan.'}, {'question': 'Hang động nào ở Việt Nam là h